# Exercices du chapitre 2

Notez que les exercices 4 et 6 ont été faits en TD.

In [ ]:
using CSV, DataFrames, Gadfly, GLM, Distributions, LinearAlgebra

# Exercice 3

In [ ]:
# Chargement des données
data = CSV.read("tuition_vs_salary.csv")

## 3 a) Illustration du salaire médian en mi-carrière en fonction des frais de scolarité

In [ ]:
plot(data, x=:Tuition, y=:Salary, Geom.point)

# Une relation linéaire semble raisonnable bien qu'il y ait des points "extremes"

## 3 b) Estimation des paramètres de la droite de régression

In [ ]:
X₁ = convert(Array{Float64},data[:Tuition])
Y = convert(Array{Float64},data[:Salary])
n = length(Y)
X = hcat(ones(n),X₁)
p = size(X,2)-1

# estimation des paramètres avec la forme matricielle
β̂ = (X'*X)\X'*Y

println("β̂ = $β̂")

## 3 c) Calcul des résidus

In [ ]:
# calcul des estimations
Ŷ = X*β̂

# calcul des résidus
e = Y-Ŷ

# stockage dans un dataframe pour un affichage facile
df = DataFrame(Estimation = Ŷ, Residu = e)

plot(df,x=:Estimation,y=:Residu, Geom.point)

#= Les hypothèses 2 et 3 semblent raisonnables. Les résidus sont bien centrés autour de 0 et la variance 
sembe relativement constante =#

## 3 d) Validation de l'hypothèse de normalité des résidus

à ne pas faire pour l'instant

## 3 e) Test de l'importance de la régression

In [ ]:
SST = sum( (Y .- mean(Y) ).^2)
SSE = sum( e.^2 )
SSR = SST - SSE

F₀ = (SSR/1) / (SSE/(n-1-1)) 

# On rejette H₀ au seuil de 5% si la valeur-p associé à F₀ est plus petite que 5%
valeurp = ccdf(FDist(p,n-p-1),F₀)
if valeurp<0.05
    println("Comme la valeur-p = $valeurp<0.05, on rejette H_0. La régression est donc significative.")
else
    println("Comme la valeur-p = $valeurp>0.05, on ne rejette pas H_0. La régression n'est donc pas significative.")
end

## 3 f) Calcul du coefficient de détermination

In [ ]:
R² = SSR/SST
println("Le coefficient de détermination est $R²")

# Exercice 5

In [ ]:
# Chargement des données
data = CSV.read("notes.csv")
first(data,5)

## 5 a) Estimation des coefficients du modèle de régression

In [ ]:
# Extraction des données
n = size(data,1)
X₁ = convert(Array{Float64},data[:CP1])
X₂ = convert(Array{Float64},data[:CP2])
X₃ = ones(n)
X₃[data[:Session].=="A2017"].=0
Y =  convert(Array{Float64},data[:Final]);
# Ici on encode la variable qualitative session par une variable indicatrice


# Construction de la matrice de structure
X = hcat(ones(n),X₁,X₂,X₃)

# Estimation des paramètres
β̂ = (X'*X)\X'*Y

println("β̂ = $β̂")

Ŷ = X*β̂
e = Y .- Ŷ

## 5 b) Validation des hypothèses 1 et 2 à l'aide des résidus

In [ ]:
# Calcul des estimations
Ŷ = X*β̂

# Calcul des résidus
e = Y - Ŷ

# stockage dans un dataframe pour un affichage facile
df = DataFrame(Estimation = Ŷ, Residu = e)

plot(df,x=:Estimation,y=:Residu, Geom.point)

#= Les hypothèses 2 et 3 semblent raisonnables. Les résidus sont bien centrés autour de 0 et la variance 
sembe relativement constante =#


## 5 c) Estimation des intervalles de confiance à 95% des coefficients

In [ ]:
# Estimation de la variance de l'erreur
σ̂² = sum(e.^2)/(n-p-1)

# Calcul de la matrice C
C = inv(X'*X)

# Calcul des intervalles de confiance
IC = [β̂-sqrt.(σ̂²*diag(C))*quantile(TDist(n-p-1),.975) β̂-sqrt.(σ̂²*diag(C))*quantile(TDist(n-p-1),.025)]

## 5 d) Y a-t-il des variables non siginificatives ?

Aucun des intervalles de confiance ne contient la valeur 0. Donc toutes les variables sont significatives

## 5 e) Prédiction de la note d'un étudiant

In [ ]:
#= Vecteur de variables explicatives correspondant à un étudaint de l'hiver ayant obtenu 13 et 15
aux intra 1 et 2 respectivement =#
x₀ = [1.0 13.0 15.0 1.0]

# Estimation de la note
Ŷ₀ = x₀*β̂
println("Ŷ₀ = $Ŷ₀")

# Intervalle de confiance à 95 %
IC = [Ŷ₀-sqrt.(σ̂²*(1 .+ x₀/(X'*X)*x₀'))*quantile(TDist(n-p-1),.975) Ŷ₀-sqrt.(σ̂²*(1 .+ x₀/(X'*X)*x₀'))*quantile(TDist(n-p-1),.025)]

println("IC = $IC")


# Exercice 7

In [ ]:
# Chargement des données
data = CSV.read("bloodpressure.csv")
first(data,5)

In [ ]:
# Extraction des données

Y = convert(Array{Float64},data[:BP])
X₁ = convert(Array{Float64},data[:Age])
X₂ = convert(Array{Float64},data[:Weight])
X₃ = convert(Array{Float64},data[:BSA])
X₄ = convert(Array{Float64},data[:Dur])
X₅ = convert(Array{Float64},data[:Pulse])
X₆ = convert(Array{Float64},data[:Stress])
n = length(Y)

X = hcat(ones(n),X₁,X₂,X₃,X₄,X₅,X₆)
p = size(X,2)-1

## 7 a) et b)

À ne pas faire

## 7 c) Calcul des valeurs propres

In [ ]:
λ = eigvals(X'X)
ϕ = sqrt.(maximum(λ)./λ)

## 7 d) Indice de multicolinéarité

In [ ]:
ϕ = sqrt.(maximum(λ)./λ)
# Il y a multicolinéarité puisqu'au moins un phi est plus grand que 30.